# Single Agent on JudgeBench with Position Swapping


## Imports

In [11]:
import pandas as pd
from datasets import load_dataset
import os
from dotenv import load_dotenv
from autogen import ConversableAgent
import re
from tqdm import tqdm

## Data

In [12]:
JudgeBench_Claude = load_dataset("ScalerLab/JudgeBench", split="claude")

df = pd.DataFrame(JudgeBench_Claude)

df_sampled = df.iloc[:270].sample(n=100, random_state=42).reset_index(drop=True)

df_final = df_sampled[["question", "response_A", "response_B", "label"]]

print(df_final.info())
print(df_final.head(1))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   question    100 non-null    object
 1   response_A  100 non-null    object
 2   response_B  100 non-null    object
 3   label       100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB
None
                                            question  \
0  Under standard temperature and pressure condit...   

                                          response_A  \
0  Let's approach this step-by-step:\n\n1) The ra...   

                                          response_B label  
0  Let's approach this step-by-step:\n\n1) The ra...   B>A  


## Config

In [13]:
load_dotenv()

api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_API_VERSION", "2023-12-01-preview")

config_list = [
    {
        "model": deployment_name,
        "api_key": api_key,
        "base_url": f"{endpoint}/openai/deployments/{deployment_name}/chat/completions?api-version={api_version}",
        "api_type": "azure",
        "api_version": api_version,  
        "temperature": 0,
        "cache_seed": 42
    }
]

## System Design

In [14]:
agent_system_message = f"""
You are an objective Evaluator-Agent.
Your task is to choose the more factually correct response to the question between Response A and Response B.
Give an explanation on your decision using about 200 words.
Always begin your output with: "As an objective Evaluator-Agent I think ..."
Always end your output with stating if Response A or B is more factually correct by using a JSON-object with the following format: {{"response": A/B}}
"""

In [15]:
initializer = ConversableAgent(
    "initializer", 
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    )

agent = ConversableAgent(
    "Evaluator-Agent", 
    llm_config={"config_list": config_list},
    system_message=agent_system_message,
    human_input_mode="NEVER",
    )

## Evaluation

In [16]:
def evaluate(question, response_A, response_B, label):

    message = f""" 
    Question: {question}

    Response A: {response_A}

    Response B: {response_B}
    """

    result = initializer.initiate_chat(agent, message=message, max_turns=1)
    result_str = str(result)

    pattern = r'"response"\s*:\s*"?([AB])"?'

    match = re.search(pattern, result_str)
    system_decision = match.group(1) if match else "X"

    ground_truth = "A" if "A>" in label else "B"

    is_correct = system_decision == ground_truth

    return {
        "system_decision": system_decision,
        "ground_truth": ground_truth,
        "is_correct": is_correct
    }

In [17]:
num_rows = 3

df_final_subset = df_final.head(num_rows)

# normal position
results_1 = []

for _, row in tqdm(df_final_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        question=row["question"],
        response_A=row["response_A"],
        response_B=row["response_B"],
        label=row["label"],
    )
    results_1.append(result)

results_1_df = pd.DataFrame(results_1)
results_1_df.to_csv('Results/single_1.csv', index=False)

# swapped position
results_2 = []

for _, row in tqdm(df_final_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        question=row["question"],
        response_A=row["response_B"], # positions swap 
        response_B=row["response_A"], # positions swap 
        label=row["label"],
    )

    # position swap back
    if result["system_decision"] == "A":
        result["system_decision"] = "B"
    elif result["system_decision"] == "B":
        result["system_decision"] = "A"

    if result["is_correct"] == True:
        result["is_correct"] = False
    elif result["is_correct"] == False:
        result["is_correct"] = True

    results_2.append(result)

results_2_df = pd.DataFrame(results_2)
results_2_df.to_csv('Results/single_2.csv', index=False)

Progress:   0%|          | 0/3 [00:00<?, ?it/s]

initializer (to Evaluator-Agent):

 
    Question: Under standard temperature and pressure conditions, compare the relative rates at which inert gases,Ar, He, and Kr diffuse through a common orifice.
(A) .1002 : .3002 : .4002
(B) .3582 : .4582 : .0582
(C) .2582 : .4998 : .3092
(D) .1582 : .6008 : .2092
(E) .1582 : .4998 : .1092
(F) .2002 : .4002 : .1092
(G) .1582 : .3998 : .2592
(H) .2502 : .4502 : .1502
(I) .2082 : .5998 : .1592
(J) .1802 : .4802 : .2802
If you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.
Let's think step by step.

    Response A: Let's approach this step-by-step:

1) The rate of diffusion of gases is inversely proportional to the square root of their molecular masses. This is known as Graham's Law.

2) We need to find the relative rates, so we'll use the formula: 
   Rate ∝ 1/√(Molecular Mass)

3) Mole

Evaluator-Agent (to initializer):

As an objective Evaluator-Agent I think both responses correctly apply Graham's Law of Diffusion to determine the relative rates of diffusion for Argon (Ar), Helium (He), and Krypton (Kr). They both accurately calculate the diffusion rates based on the molecular masses of the gases. However, the final conclusions drawn by each response differ.

Response A concludes that the closest match to the normalized rates is option D (0.1582 : 0.6008 : 0.2092), while Response B concludes that the closest match is option E (0.1582 : 0.4998 : 0.1092). Upon reviewing the calculations, Response A's normalization of the rates is slightly more accurate, as it aligns better with the expected diffusion rates based on the molecular weights provided.

In terms of factual correctness, Response A provides a more accurate representation of the diffusion rates and correctly identifies the closest option. Therefore, I conclude that Response A is more factually correct.

{"resp

Progress:  33%|███▎      | 1/3 [00:04<00:08,  4.32s/it]

initializer (to Evaluator-Agent):

 
    Question: The total cost of producing x cameras is C(x) = 2 + x^3. What is the average cost if 10 cameras are made? What is the marginal cost of producing 10 cameras?
(A) Average Cost: $1000/unit, Marginal Cost: $100
(B) Average Cost: $300/unit, Marginal Cost: $100
(C) Average Cost: $300/unit, Marginal Cost: $1000
(D) Average Cost: $102/unit, Marginal Cost: $301
(E) Average Cost: $500/unit, Marginal Cost: $1000
(F) Average Cost: $100/unit, Marginal Cost: $100
(G) Average Cost: $200/unit, Marginal Cost: $300
(H) Average Cost: $2/unit, Marginal Cost: $30
(I) Average Cost: $1020/unit, Marginal Cost: $299
(J) Average Cost: $100/unit, Marginal Cost: $300
If you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.
Let's think step by step.

    Response A: Let's approach this step-by-step:

1) 

...

Progress:  33%|███▎      | 1/3 [00:04<00:09,  4.94s/it]

initializer (to Evaluator-Agent):

 
    Question: The total cost of producing x cameras is C(x) = 2 + x^3. What is the average cost if 10 cameras are made? What is the marginal cost of producing 10 cameras?
(A) Average Cost: $1000/unit, Marginal Cost: $100
(B) Average Cost: $300/unit, Marginal Cost: $100
(C) Average Cost: $300/unit, Marginal Cost: $1000
(D) Average Cost: $102/unit, Marginal Cost: $301
(E) Average Cost: $500/unit, Marginal Cost: $1000
(F) Average Cost: $100/unit, Marginal Cost: $100
(G) Average Cost: $200/unit, Marginal Cost: $300
(H) Average Cost: $2/unit, Marginal Cost: $30
(I) Average Cost: $1020/unit, Marginal Cost: $299
(J) Average Cost: $100/unit, Marginal Cost: $300
If you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.
Let's think step by step.

    Response A: Let's approach this step-by-step:

1) 

Progress:  67%|██████▋   | 2/3 [00:09<00:04,  4.73s/it]

initializer (to Evaluator-Agent):

 
    Question: As of 2013, share of people in the India who think political parties are corrupt is
(A) 86%
(B) 26%
(C) 50%
(D) 66%
(E) 90%
(F) 70%
(G) 46%
(H) 10%
(I) 76%
(J) 30%
If you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.
Let's think step by step.

    Response A: Let's approach this step-by-step:

1. The question asks about data from 2013, so we need to focus on information from that specific year.

2. The question is about the perception of corruption in political parties in India.

3. Unfortunately, I don't have access to precise data for this specific question from 2013.

4. However, based on general knowledge about corruption perceptions in India, we can make an educated guess.

5. India has historically had high perceived levels of corruption in politics.

6. The options

Progress: 100%|██████████| 3/3 [00:13<00:00,  4.38s/it]
